In [1]:
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup as bs

In [2]:
iebc_url = "https://www.iebc.or.ke/registration/?where"

## Let's create a function to clean the scraped data and return a clean dataframe

In [3]:
def convert_to_dataframe(junkdata, data_category, optional_val=None, optional_val2=None):
    store = list()
    for record in junkdata:
        # Exctracting the code
        code = record.get('value')
        if code:
            # Exctracting the name
            name = record.text
            
            # Storing the records in a temporary list
            
            # This means that the request is for wards
            if optional_val2:
                store.append({f"Constituency_Code": optional_val2, f"{data_category}_Code": code, f"{data_category}_Name": name})
            # This means that the request is for constituencies
            elif optional_val:
                store.append({f"County_Code":optional_val, f"{data_category}_Code": code, f"{data_category}_Name": name})
            # This means that the request is for counties
            else:
                store.append({f"{data_category}_Code": code, f"{data_category}_Name": name})
    df = pd.DataFrame(store)
    return df

## Getting the initial data from the page

In [4]:
initial_page = requests.get(iebc_url)
if initial_page.status_code == 200:
    soup = bs(initial_page.content, 'html.parser')
else:
    print("The page did not load!")

## Getting the counties

In [5]:
counties = soup.find_all('select', id='county')[0].find_all('option')

# Clean the data and have it a dataframe
counties_df = convert_to_dataframe(junkdata=counties, data_category='County')
counties_df.sort_values(by='County_Code', inplace=True)
counties_df.reset_index(drop=True, inplace=True)
# Save to csv file
if len(counties_df.values) != 0:
    counties_df.to_csv('counties.csv', index=False)
counties_df.head()

,County_Code,County_Name
0,001,MOMBASA
1,002,KWALE
2,003,KILIFI
3,004,TANA RIVER
4,005,LAMU


## Making a post request to get the contituencies in each county

In [6]:
# Make the request
def get_constituencies():
    county_post_url = "https://www.iebc.or.ke/registration/show_const.php"
    unionized_df = pd.DataFrame(columns=['County_Code', 'Constituency_Code', 'Constituency_Name'])
    for county_id in counties_df.iloc[:,0]:
        county_payload = {"cid":county_id}
        constituencies_reponse = requests.post(county_post_url, data=county_payload)
        
        if constituencies_reponse.status_code == 200:
            constituencies_soup = bs(constituencies_reponse.content, 'html.parser')
            constituencies = constituencies_soup.find_all('select', id='constituency')[0].find_all('option')
            constituencies_df = convert_to_dataframe(junkdata=constituencies, data_category='Constituency', optional_val=county_id)
            unionized_df = pd.concat([unionized_df, constituencies_df])
        else:
            print(f"Did not retrive the data for County {county_id}")
    unionized_df.sort_values(by=['County_Code','Constituency_Code'], inplace=True)
    unionized_df.reset_index(drop=True, inplace=True)
    # Save to csv file
    if len(unionized_df.values) != 0:
        unionized_df.to_csv('constituencies.csv', index=False)
    return unionized_df
constituencies_df = get_constituencies()

In [7]:
constituencies_df

,County_Code,Constituency_Code,Constituency_Name
0,001,001,CHANGAMWE
1,001,002,JOMVU
2,001,003,KISAUNI
3,001,004,NYALI
4,001,005,LIKONI
...,...,...,...
286,047,287,MAKADARA
287,047,288,KAMUKUNJI
288,047,289,STAREHE
289,047,290,MATHARE


## Making a post request to get the list of wards

In [8]:
def get_wards():
    constituency_post_url = "https://www.iebc.or.ke/registration/show_wards.php"
    unionized_df = pd.DataFrame(columns=['Constituency_Code', 'Ward_Code', 'Ward_Name'])
    for constituency_id in constituencies_df.iloc[:,1]:
        constituencies_payload = {"ccid":constituency_id}
        wards_reponse = requests.post(constituency_post_url, data=constituencies_payload)
        
        if wards_reponse.status_code == 200:
            wards_soup = bs(wards_reponse.content, 'html.parser')
            wards = wards_soup.find_all('select', id='wards')[0].find_all('option')
            wards_df = convert_to_dataframe(junkdata=wards, data_category='Ward', optional_val2=constituency_id)
            unionized_df = pd.concat([unionized_df, wards_df])
        else:
            print(f"Did not retrive the data for County {constituency_id}")
    unionized_df.sort_values(by=['Constituency_Code', 'Ward_Code'], inplace=True)
    unionized_df.reset_index(drop=True, inplace=True)
    # Save to csv file
    if len(unionized_df.values) != 0:
        unionized_df.to_csv('wards.csv', index=False)
    return unionized_df
wards_df = get_wards()

In [9]:
wards_df

,Constituency_Code,Ward_Code,Ward_Name
0,001,0001,PORT REITZ
1,001,0002,KIPEVU
2,001,0003,AIRPORT
3,001,0004,CHANGAMWE
4,001,0005,CHAANI
...,...,...,...
1449,290,1450,KIAMAIKO
1450,291,5000,TANZANIA
1451,291,5001,UGANDA
1452,291,5002,RWANDA


## Making a post request to get the list of polling stations

In [10]:
def get_polling_stations():
    ward_url = "https://www.iebc.or.ke/registration/show_stations.php"
    polling_stations = list()
    
    for ward_id in wards_df.iloc[:, 1]:
        ward_payload = {"wardid": ward_id}
        station_reponse = requests.post(ward_url, data=ward_payload)
        
        if station_reponse.status_code == 200:
            station_soup = bs(station_reponse.content, 'html.parser')
            stations = station_soup.find('table').find_all('tr')
            
            for station in stations:
                polling_station = station.find('td').text
                polling_stations.append({"Ward_Code": ward_id, "Station_Name": polling_station})
                
    stations_df = pd.DataFrame(polling_stations)
    stations_df.sort_values(by='Ward_Code', inplace=True)
    stations_df.reset_index(drop=True, inplace=True)
    stations_df.to_csv("polling_stations.csv", index=False)
    return stations_df

polling_stations_df = get_polling_stations()

In [11]:
polling_stations_df

,Ward_Code,Station_Name
0,0001,BOMU PRIMARY SCHOOL
1,0001,MWIJABU PRIMARY SCHOOL
2,0001,LILONGWE GARDEN
3,0002,UMOJA PRIMARY SCHOOL
4,0002,KISUMU NDOGO GROUND
...,...,...
24613,5000,ARUSHA
24614,5000,DAR ES SALAAM
24615,5001,KAMPALA
24616,5002,KIGALI


## Getting registration officers for each consituency

In [12]:
def get_registration_officers():
    contact_url = "https://www.iebc.or.ke/registration/show_contacts.php"
    officers_list = list()
    for constituency_id in constituencies_df.iloc[:, 1]:
        officer_payload = {'ccid': constituency_id}
        officer_response = requests.post(contact_url, data=officer_payload)

        if officer_response.status_code == 200:
            officer_soup = bs(officer_response.content, 'html.parser')
            rows = officer_soup.find('table').find_all('td')
            name = rows[0].text
            email = rows[1].text
            officers_list.append({"Constituency_Code": constituency_id, "Officer_Name": name, "Officer_Email": email})
    officers_df = pd.DataFrame(officers_list)
    officers_df.sort_values(by='Constituency_Code', inplace=True)
    officers_df.reset_index(drop=True, inplace=True)
    officers_df.to_csv("registration_officers.csv", index=False)
    return officers_df

registration_officers_df = get_registration_officers()

In [13]:
registration_officers_df

,Constituency_Code,Officer_Name,Officer_Email
0,001,DOUGLAS KARISA NEEMA,NKarisa@IEBC.OR.KE
1,002,D. KOMBE HILARY,HKombe@IEBC.OR.KE
2,003,AISHA SENGE ABUBAKAR,ASenge@IEBC.OR.KE
3,004,EISHA OMAR MOHAMED,EOmar@IEBC.OR.KE
4,005,GULIYA HUSSEIN ABIDWAHID,AHussein@IEBC.OR.KE
...,...,...,...
286,287,SAHARA IBRAHIM MAALIM,ISahara@IEBC.OR.KE
287,288,WANDEO AWUOR PAMELA,PWandeo@IEBC.OR.KE
288,289,MARVIN MUNGAH KARANJA,MKaranja@IEBC.OR.KE
289,290,MUIGAI KAMAU PETER,PMuigai@IEBC.OR.KE


### Saving to a excel file

In [14]:
filename = 'iebc_records.xlsx'

# Writing the dataframe to different sheets on the same file
with pd.ExcelWriter(filename) as writer:
    # Sheet 1: Counties
    counties_df.to_excel(writer, sheet_name='Counties', index=False)
    # Sheet 2: Constituencies
    constituencies_df.to_excel(writer, sheet_name='Constituencies', index=False)
    # Sheet 3: Wards
    wards_df.to_excel(writer, sheet_name='Wards', index=False)
    # Sheet 4: Polling Stations
    polling_stations_df.to_excel(writer, sheet_name='Polling_Stations', index=False)
    # Sheet 5: Registration officers
    registration_officers_df.to_excel(writer, sheet_name='Registration_Officers', index=False)

## Exporting the dataframes to a database

#### Connecting to the database

In [15]:
conn = sqlite3.connect('iebcrecords.db')
cursor = conn.cursor()

#### Enabling foreign key support

In [16]:
cursor.execute("PRAGMA foreign_keys = ON")

#### Creating tables for the various dataframes

In [17]:
# Counties table
cursor.execute('CREATE TABLE IF NOT EXISTS counties (county_code VARCHAR(10) PRIMARY KEY, county_name TEXT NOT NULL)')

# Constituencies table
cursor.execute('''
CREATE TABLE IF NOT EXISTS constituencies (
    county_code VARCHAR(10),
    constituency_code VARCHAR(10) PRIMARY KEY,
    constituency_name TEXT NOT NULL,
    FOREIGN KEY (county_code) REFERENCES counties (county_code)
)
''')

# Wards table
cursor.execute('''
CREATE TABLE IF NOT EXISTS wards (
    constituency_code VARCHAR(10),
    ward_code VARCHAR(10) PRIMARY KEY,
    ward_name TEXT NOT NULL,
    FOREIGN KEY (constituency_code) REFERENCES constituencies (constituency_code)
)
''')

# Polling stations
cursor.execute('''
CREATE TABLE IF NOT EXISTS polling_stations (
    ward_code VARCHAR(10),
    station_name TEXT NOT NULL,
    FOREIGN KEY (ward_code) REFERENCES wards (ward_code)
)
''')

# Registration officers
cursor.execute('''
CREATE TABLE IF NOT EXISTS registration_officers (
    constituency_code VARCHAR(10),
    officer_name TEXT NOT NULL,
    officer_email VARCHAR(20) NOT NULL,
    FOREIGN KEY (constituency_code) REFERENCES constituencies (constituency_code)
)
''')
conn.commit()

#### Inserting Data from DataFrames into Tables

In [18]:
# It would be easier to use the built in df.to_sql() but that would make it a 
# little bit tough to ensure data integrity incase of reruns. So we are going to write custom insert queries
try:
    # Insert Counties
    print("\n[+] Processing Counties...")
    counties = counties_df.values
    for i, county in enumerate(counties):
        query = f"INSERT OR IGNORE INTO counties {tuple(counties_df.columns.to_list())} VALUES (?, ?);"
        cursor.execute(query, county)
        print(f"Inserted {i+1}/{len(counties)} records\r", end="")
    print("\n[+] Succefully processed counties.")
        
    # Insert into Constituencies
    print("\n[+] Processing Constituencies...")
    constituencies = constituencies_df.values
    for i, constituency in enumerate(constituencies):
        query = f"INSERT OR IGNORE INTO constituencies {tuple(constituencies_df.columns.to_list())} VALUES (?, ?, ?);"
        cursor.execute(query, constituency)
        print(f"Inserted {i+1}/{len(constituencies)} records\r", end="")
    print("\n[+] Successfully processed constituencies")

    # Insert into Wards
    print("\n[+] Processing Wards...")
    wards = wards_df.values
    for i, ward in enumerate(wards):
        query = f"INSERT OR IGNORE INTO wards {tuple(wards_df.columns.to_list())} VALUES (?, ?, ?);"
        cursor.execute(query, ward)
        print(f"Inserted {i+1}/{len(wards)} records\r", end="")
    print("\n[+] Successfully processed wards.")

    # Insert into Polling Stations
    print("\n[+] Processing Polling Stations...")
    stations = polling_stations_df.values
    for i, station in enumerate(stations):
        query = f"INSERT OR IGNORE INTO polling_stations {tuple(polling_stations_df.columns.to_list())} VALUES (?, ?);"
        cursor.execute(query, station)
        print(f"Inserted {i+1}/{len(stations)} records\r", end="")
    print("\n[+] Successfully processed polling stations.")

    # Insert into Registrars
    print("\n[+] Processing Registars...")
    registrars = registration_officers_df.values
    for i, registrar in enumerate(registrars):
        query = f"INSERT OR IGNORE INTO registration_officers {tuple(registration_officers_df.columns.to_list())} VALUES (?, ?, ?);"
        cursor.execute(query, registrar)
        print(f"Inserted {i+1}/{len(registrars)} records\r", end="")
    print("\n[+] Successfully processed registrars.")
    conn.commit()

    # Check if the view already exists
    cursor.execute("SELECT name FROM sqlite_master WHERE type='view' AND name='POLLING_STATION_VIEW';")
    view_exists = cursor.fetchone()

    # Create the view if it doesn't exist
    if not view_exists:
        cursor.execute("""
        CREATE VIEW POLLING_STATION_VIEW AS
        SELECT 
            counties.county_name, 
            constituencies.constituency_name, 
            wards.ward_name, 
            polling_stations.station_name,
            registration_officers.officer_name, 
            registration_officers.officer_email 
        FROM 
            counties
        INNER JOIN 
            constituencies ON counties.county_code = constituencies.county_code
        INNER JOIN 
            wards ON constituencies.constituency_code = wards.constituency_code
        INNER JOIN 
            polling_stations ON wards.ward_code = polling_stations.ward_code
        INNER JOIN 
            registration_officers ON constituencies.constituency_code = registration_officers.constituency_code;
        """)
        print("View POLLING_STATION_VIEW created.")
    else:
        print("View POLLING_STATION_VIEW already exists.")
    conn.commit()
    conn.close()
    print("[+] Done")
except KeyboardInterrupt:
    conn.rollback()
    conn.close()
except Exception as e:
    print(e.args[0])
    conn.rollback()
    conn.close()


[+] Processing Counties...
Inserted 48/48 records
[+] Succefully processed counties.

[+] Processing Constituencies...
Inserted 291/291 records
[+] Successfully processed constituencies

[+] Processing Wards...
Inserted 1454/1454 records
[+] Successfully processed wards.

[+] Processing Polling Stations...
Inserted 24618/24618 records
[+] Successfully processed polling stations.

[+] Processing Registars...
Inserted 291/291 records
[+] Successfully processed registrars.
View POLLING_STATION_VIEW created.
[+] Done
